<a href="https://colab.research.google.com/github/UoA-eResearch/mussels/blob/main/measure_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In Colab, be sure to select 'T4 GPU' under 'Edit'->'Notebook Settings'->'Hardware accelerator'.

In [1]:
!nvidia-smi

Thu Oct  5 02:01:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone https://github.com/UoA-eResearch/mussels.git

Cloning into 'mussels'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 79 (delta 17), reused 21 (delta 9), pack-reused 46
Receiving objects: 100% (79/79), 43.53 MiB | 33.92 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [3]:
cd mussels

/content/mussels


In [4]:
ls

EDA.html   LICENSE            README.md         test.png
EDA.ipynb  measure_all.ipynb  requirements.txt  util.py*


In [5]:
!pip install -qr requirements.txt
%load_ext autotime
from util import *

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 75.4 MB/s eta 0:00:00


In [6]:
!wget https://www.dropbox.com/sh/6khb99svor0buhr/AAC9WmmSwW7v1poWAsb6wOzJa?dl=1 -O images.zip

--2023-10-05 02:01:55--  https://www.dropbox.com/sh/6khb99svor0buhr/AAC9WmmSwW7v1poWAsb6wOzJa?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /sh/dl/6khb99svor0buhr/AAC9WmmSwW7v1poWAsb6wOzJa [following]
--2023-10-05 02:01:55--  https://www.dropbox.com/sh/dl/6khb99svor0buhr/AAC9WmmSwW7v1poWAsb6wOzJa
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc046d3966e001fb62baebc91c51.dl-au.dropboxusercontent.com/zip_download_get/BobbjmHCKKCU9juTUc_pjEzRix83rPF9cgQ7AF0XnRV1_4iD3odiOV3u0fTj_xavyjEiatcFXI6ewPPS4pjALHd20aC0dw85OdUk_VrxcaKCIQ?dl=1# [following]
--2023-10-05 02:01:56--  https://uc046d3966e001fb62baebc91c51.dl-au.dropboxusercontent.com/zip_download_get/BobbjmHCKKCU9juTUc_pjEzRix83rPF9cgQ7AF0XnRV1_4iD3odiOV3u0fTj_xavyjEiatcFXI6ewPPS4pj

In [7]:
!unzip -q images.zip -d images

mapname:  conversion of  failed


In [8]:
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

--2023-10-05 02:02:28--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.180.84, 3.163.180.121, 3.163.180.70, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.180.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h_4b8939.pth’

sam_vit_h_4b8939.pt 100%[===================>]   2.39G   246MB/s    in 10s     

2023-10-05 02:02:38 (245 MB/s) - ‘sam_vit_h_4b8939.pth’ saved [2564550879/2564550879]



In [9]:
sam = load_SAM()

In [10]:
files = sorted(glob("images/**/*.JPEG", recursive=True))
len(files)

200

In [ ]:
results = []
# 259/259 [5:09:14<00:00, 71.64s/it]
for f in tqdm(files):
    print(f)
    df = measure_mussels_in_image(sam, f, plot=True)
    os.makedirs("results/" + os.path.dirname(f), exist_ok=True)
    df.to_csv("results/" + f + ".csv")
    stats = df.length_cm.describe()
    print(stats)
    stats["filename"] = f
    results.append(stats)
    pd.DataFrame(results).to_csv("results.csv")

  0%|          | 0/200 [00:00<?, ?it/s]

images/D1/IMG_7498.JPEG
count    214.000000
mean       1.899287
std        0.268180
min        1.073705
25%        1.753183
50%        1.881578
75%        2.031530
max        2.879656
Name: length_cm, dtype: float64
images/D1/IMG_7499.JPEG
